In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #隔離備卡

import torch
print(f"Built with CUDA: {torch.version.cuda}")
print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")

Built with CUDA: 12.8
GPU 名稱: NVIDIA GeForce RTX 5070 Ti


In [2]:
!pip install datasets

In [3]:
!pip install -U ipywidgets

In [4]:
from datasets import load_dataset

# 使用已轉換好的版本
dataset = load_dataset(
    "FinanceInc/auditor_sentiment",  # 這是轉換後的版本
    #split="train"
)

print(dataset)

print("訓練集：", dataset['train'])
print("測試集：", dataset['test'])

# 查看資料集的欄位名稱與類型
print("訓練集欄位：", dataset["train"].column_names)
print("測試集欄位：", dataset["test"].column_names)

C:\ProgramData\anaconda3\envs\torch310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3877
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 969
    })
})
訓練集： Dataset({
    features: ['sentence', 'label'],
    num_rows: 3877
})
測試集： Dataset({
    features: ['sentence', 'label'],
    num_rows: 969
})
訓練集欄位： ['sentence', 'label']
測試集欄位： ['sentence', 'label']


In [5]:
# 查看訓練集中前幾筆數據
print("\n訓練集前5筆：")
print(dataset["train"][:5])

# 查看測試集中前幾筆數據
print("\n測試集前5筆：")
print(dataset["test"][:5])

# 查看資料集中有哪些 Keys
print("\n資料集的 keys：", dataset.keys())


訓練集前5筆：
{'sentence': ["Altia 's operating profit jumped to EUR 47 million from EUR 6.6 million .", 'The agreement was signed with Biohit Healthcare Ltd , the UK-based subsidiary of Biohit Oyj , a Finnish public company which develops , manufactures and markets liquid handling products and diagnostic test systems .', 'Kesko pursues a strategy of healthy , focused growth concentrating on sales and services to consumer-customers .', 'Vaisala , headquartered in Helsinki in Finland , develops and manufactures electronic measurement systems for meteorology , environmental sciences , traffic and industry .', 'Also , a six-year historic analysis is provided for these markets .'], 'label': [2, 2, 2, 1, 1]}

測試集前5筆：
{'sentence': ["TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .", 'STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) ra

In [6]:
from collections import Counter # 計算列表中每個元素的出現次數

# label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
# 計算訓練集標籤出現次數
label_counts_train = Counter(dataset["train"]["label"])
print("\n訓練集標籤分佈：", label_counts_train)

# 計算測試集標籤出現次數
label_counts_test = Counter(dataset["test"]["label"])
print("測試集標籤分佈：", label_counts_test)


訓練集標籤分佈： Counter({1: 2320, 2: 1077, 0: 480})
測試集標籤分佈： Counter({1: 559, 2: 286, 0: 124})


In [7]:
import pandas as pd  # 結構化資料處理套件

# 轉換為 DataFrame
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

print("\n訓練集前5筆：")
print(df_train.head())

print("\n測試集前5筆：")
print(df_test.head())


訓練集前5筆：
                                            sentence  label
0  Altia 's operating profit jumped to EUR 47 mil...      2
1  The agreement was signed with Biohit Healthcar...      2
2  Kesko pursues a strategy of healthy , focused ...      2
3  Vaisala , headquartered in Helsinki in Finland...      1
4  Also , a six-year historic analysis is provide...      1

測試集前5筆：
                                            sentence  label
0  TeliaSonera TLSN said the offer is in line wit...      2
1  STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMEN...      2
2  Clothing retail chain Sepp+ñl+ñ 's sales incre...      2
3  Lifetree was founded in 2000 , and its revenue...      2
4  Nordea Group 's operating profit increased in ...      2


In [8]:
# 檢查空值
print("\n訓練集空值：", df_train.isnull().sum())
print("測試集空值：", df_test.isnull().sum())

# 刪除重複值（如果有的話）
df_train.drop_duplicates(subset=["sentence"], inplace=True)
df_test.drop_duplicates(subset=["sentence"], inplace=True)

print("\n訓練集大小：", df_train.shape)
print("測試集大小：", df_test.shape)


訓練集空值： sentence    0
label       0
dtype: int64
測試集空值： sentence    0
label       0
dtype: int64

訓練集大小： (3872, 2)
測試集大小： (969, 2)


In [9]:
train_val_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = dataset["test"]
print("\n資料集劃分：")
print(f"訓練集大小：{len(train_dataset)}")
print(f"驗證集大小：{len(val_dataset)} （從訓練集切出，用於訓練過程中的評估）")
print(f"測試集大小：{len(test_dataset)} （保留到最後才使用，避免資訊洩漏）")


資料集劃分：
訓練集大小：3101
驗證集大小：776 （從訓練集切出，用於訓練過程中的評估）
測試集大小：969 （保留到最後才使用，避免資訊洩漏）


In [10]:
from transformers import AutoTokenizer  # AutoTokenizer 可以根據指定的模型名稱，自動選擇和載入與該模型相匹配的分詞器

# 使用BERT Tokenizer 進行Tokenization(分詞)
#model_name = "bert-base-uncased"  # 請修改這裡！(預設)
#model_name = "distilbert-base-uncased"  # BERT 的蒸餾版本，速度快 60%，模型小 40%
#model_name = "roberta-base"  # 改進的 BERT，通常效果更好
#model_name = "albert-base-v2"  # 參數共享技術，模型很小但效果不錯
model_name = "ProsusAI/finbert"  # 專門在金融文本上預訓練，理解金融術語
#model_name = "bert-base-chinese"  # 支援中文

print(f"\n使用的模型：{model_name}")
# ============================================================================

from transformers import AutoTokenizer  # AutoTokenizer 可以根據指定的模型名稱，自動選擇和載入與該模型相匹配的分詞器

# 使用指定的 Tokenizer 進行 Tokenization(分詞)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 定義 Tokenize 函數，用於將句子轉換為 Token
def tokenize_function(example):
    # 使用 tokenizer 將每個句子進行分詞
    # 設定 padding="max_length" 確保所有句子補齊至固定長度
    # 設定 truncation=True 以截斷超過最大長度的句子
    return tokenizer(example["sentence"], padding="max_length", truncation=True)

# 對訓練集、驗證集和測試集進行 Tokenize
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


使用的模型：ProsusAI/finbert


Map: 100%|██████████| 776/776 [00:00<00:00, 6455.77 examples/s]


In [11]:
# 查看 Tokenized 資料
print("\nTokenized 訓練集第一筆：")
print(tokenized_train[:1])

print("\nTokenized 驗證集第一筆：")
print(tokenized_val[:1])

print("\nTokenized 測試集第一筆：")
print(tokenized_test[:1])
# 確認 tokenized_datasets 是否包含需要的欄位
print("\n訓練集特徵：", tokenized_train.features)
print("驗證集特徵：", tokenized_val.features)
print("測試集特徵：", tokenized_test.features)


Tokenized 訓練集第一筆：
{'sentence': ["The financial impact is estimated to be some 1.5 MEUR annual improvement in the division 's result , starting from fiscal year 2007 ."], 'label': [2], 'input_ids': [[101, 1996, 3361, 4254, 2003, 4358, 2000, 2022, 2070, 1015, 1012, 1019, 2033, 3126, 3296, 7620, 1999, 1996, 2407, 1005, 1055, 2765, 1010, 3225, 2013, 10807, 2095, 2289, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
import sys, torch, platform
print("Py:", sys.executable)
print("PyTorch:", torch.__version__)
print("Built with CUDA:", torch.version.cuda)
print("cuda.is_available:", torch.cuda.is_available())
print("device_count:", torch.cuda.device_count())

Py: C:\ProgramData\anaconda3\envs\torch310\python.exe
PyTorch: 2.9.0+cu128
Built with CUDA: 12.8
cuda.is_available: True
device_count: 1


In [13]:
import torch  # 深度學習框架

# 如果有GPU就用GPU，沒有GPU用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
from transformers import AutoModelForSequenceClassification  # 序列分類模型

# 載入模型（使用作業區域 1 選擇的模型）
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)  # 將模型移到對應裝置

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
from transformers import TrainingArguments  # 包裝訓練參數

training_args = TrainingArguments(
    output_dir="./results",          # 訓練結果的儲存位置，包括模型檔案和預測結果。
    eval_strategy="epoch",     # 設置評估策略為每個訓練輪次（epoch）結束後進行一次評估。
    learning_rate=2e-5,              # 設置學習率，這裡使用 2e-5，通常較小的學習率更適合微調預訓練模型。
    per_device_train_batch_size=4,   # 設定每個裝置（如每張 GPU）的訓練批次大小為 4。
    per_device_eval_batch_size=4,    # 設定每個裝置的評估批次大小為 4。
    num_train_epochs=3,              # 訓練輪次設定為 3，模型將完整遍歷訓練集三次。
    weight_decay=0.01,               # 設置權重衰減（L2正則化）參數，這裡為 0.01，用於防止過擬合。
    logging_dir="./logs",            # 日誌存放目錄，用於儲存 TensorBoard 或其他日誌。
    logging_steps=10,                # 設置每 10 個步驟記錄一次訓練信息，便於監控訓練過程。
    report_to="none",                 # 不將訓練日誌發送到任何平臺（如 TensorBoard），可以改為 "tensorboard" 以啟用。
)

In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids  # 取得真實值的id
    preds = np.argmax(pred.predictions, axis=1)  # 預測值找到最高機率的索引
    accuracy = accuracy_score(labels, preds)  # 計算準確率
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")  # 計算其他指標
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,  # 使用訓練集
    eval_dataset=tokenized_val,     # 使用驗證集（從訓練集切出來的）
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.372800,0.523612,0.858247,0.860621,0.858247,0.853417
2,0.314000,0.574850,0.880155,0.883646,0.880155,0.881095
3,0.076100,0.630002,0.878866,0.879090,0.878866,0.878944


TrainOutput(global_step=2328, training_loss=0.30931444266487773, metrics={'train_runtime': 205.2543, 'train_samples_per_second': 45.324, 'train_steps_per_second': 11.342, 'total_flos': 2447744125123584.0, 'train_loss': 0.30931444266487773, 'epoch': 3.0})

In [ ]:
###bert-base-uncased ###
#=== 驗證集評估結果 ===
#{'eval_loss': 0.7858926653862, 'eval_accuracy': 0.854381443298969, 'eval_precision': 0.8570445422160716, 'eval_recall': 0.854381443298969, 'eval_f1': 0.8552248211724083, 'eval_runtime': 4.9729, 'eval_samples_per_second': 156.045, 'eval_steps_per_second': 39.011, 'epoch': 3.0}
#=== 測試集評估結果（最終評估）===
#{'eval_loss': 0.7858926653862, 'eval_accuracy': 0.854381443298969, 'eval_precision': 0.8570445422160716, 'eval_recall': 0.854381443298969, 'eval_f1': 0.8552248211724083, 'eval_runtime': 4.9729, 'eval_samples_per_second': 156.045, 'eval_steps_per_second': 39.011, 'epoch': 3.0}

###distilbert-base-uncased ###
#=== 驗證集評估結果 ===
#{'eval_loss': 0.6798608303070068, 'eval_accuracy': 0.8595360824742269, 'eval_precision': 0.8602398356540399, 'eval_recall': 0.8595360824742269, 'eval_f1': 0.8598233041398702, 'eval_runtime': 2.7743, 'eval_samples_per_second': 279.715, 'eval_steps_per_second': 69.929, 'epoch': 3.0}
#=== 測試集評估結果（最終評估）===
#{'eval_loss': 0.6542311906814575, 'eval_accuracy': 0.8524251805985552, 'eval_precision': 0.8533467059016294, 'eval_recall': 0.8524251805985552, 'eval_f1': 0.852773891686464, 'eval_runtime': 3.7122, 'eval_samples_per_second': 261.032, 'eval_steps_per_second': 65.46, 'epoch': 3.0}

###roberta-base ###
#=== 驗證集評估結果 ===
#{'eval_loss': 0.7835732698440552, 'eval_accuracy': 0.8556701030927835, 'eval_precision': 0.8606610119547271, 'eval_recall': 0.8556701030927835, 'eval_f1': 0.8566794745799603, 'eval_runtime': 4.8835, 'eval_samples_per_second': 158.904, 'eval_steps_per_second': 39.726, 'epoch': 3.0}
#=== 測試集評估結果（最終評估）===
#{'eval_loss': 0.6186093091964722, 'eval_accuracy': 0.8782249742002064, 'eval_precision': 0.881678175780643, 'eval_recall': 0.8782249742002064, 'eval_f1': 0.8790593474119579, 'eval_runtime': 6.4364, 'eval_samples_per_second': 150.551, 'eval_steps_per_second': 37.754, 'epoch': 3.0}

###albert-base-v2 ###
#=== 驗證集評估結果 ===
#{'eval_loss': 0.8814711570739746, 'eval_accuracy': 0.8389175257731959, 'eval_precision': 0.839535683413993, 'eval_recall': 0.8389175257731959, 'eval_f1': 0.8388740180733086, 'eval_runtime': 5.9498, 'eval_samples_per_second': 130.425, 'eval_steps_per_second': 32.606, 'epoch': 3.0}
#=== 測試集評估結果（最終評估）===
#{'eval_loss': 0.8491798043251038, 'eval_accuracy': 0.8359133126934984, 'eval_precision': 0.8355952897585286, 'eval_recall': 0.8359133126934984, 'eval_f1': 0.8356977619576588, 'eval_runtime': 7.5359, 'eval_samples_per_second': 128.584, 'eval_steps_per_second': 32.245, 'epoch': 3.0}

###ProsusAI/finbert ###
#=== 驗證集評估結果 ===
#{'eval_loss': 0.6300020217895508, 'eval_accuracy': 0.8788659793814433, 'eval_precision': 0.8790897241111448, 'eval_recall': 0.8788659793814433, 'eval_f1': 0.8789437872905964, 'eval_runtime': 5.0041, 'eval_samples_per_second': 155.072, 'eval_steps_per_second': 38.768, 'epoch': 3.0}
#=== 測試集評估結果（最終評估）===
#{'eval_loss': 0.641608476638794, 'eval_accuracy': 0.8833849329205367, 'eval_precision': 0.8837511917941562, 'eval_recall': 0.8833849329205367, 'eval_f1': 0.8835018141131376, 'eval_runtime': 6.5479, 'eval_samples_per_second': 147.987, 'eval_steps_per_second': 37.111, 'epoch': 3.0}

###bert-base-chinese ###
#=== 驗證集評估結果 ===
#{'eval_loss': 0.670444130897522, 'eval_accuracy': 0.75, 'eval_precision': 0.7408298535042016, 'eval_recall': 0.75, 'eval_f1': 0.7430804194826176, 'eval_runtime': 5.1189, 'eval_samples_per_second': 151.596, 'eval_steps_per_second': 37.899, 'epoch': 3.0}
#=== 測試集評估結果（最終評估）===
#{'eval_loss': 0.7372995615005493, 'eval_accuracy': 0.7275541795665634, 'eval_precision': 0.7154559882946165, 'eval_recall': 0.7275541795665634, 'eval_f1': 0.7187897162310931, 'eval_runtime': 6.5224, 'eval_samples_per_second': 148.565, 'eval_steps_per_second': 37.256, 'epoch': 3.0}

#>>>>ProsusAI/finbert的效果最好，挑這個

In [18]:
# 評估模型在驗證集的表現

print("\n=== 驗證集評估結果 ===")
val_result = trainer.evaluate(eval_dataset=tokenized_val)
print(val_result)


=== 驗證集評估結果 ===


{'eval_loss': 0.6300020217895508, 'eval_accuracy': 0.8788659793814433, 'eval_precision': 0.8790897241111448, 'eval_recall': 0.8788659793814433, 'eval_f1': 0.8789437872905964, 'eval_runtime': 5.2288, 'eval_samples_per_second': 148.407, 'eval_steps_per_second': 37.102, 'epoch': 3.0}


In [19]:
# 評估模型在測試集的表現（最終評估）
print("\n=== 測試集評估結果（最終評估）===")
test_result = trainer.evaluate(eval_dataset=tokenized_test)
print(test_result)


=== 測試集評估結果（最終評估）===
{'eval_loss': 0.641608476638794, 'eval_accuracy': 0.8833849329205367, 'eval_precision': 0.8837511917941562, 'eval_recall': 0.8833849329205367, 'eval_f1': 0.8835018141131376, 'eval_runtime': 6.5713, 'eval_samples_per_second': 147.458, 'eval_steps_per_second': 36.979, 'epoch': 3.0}


In [20]:
# 先展示測試集中的句子，讓學生了解有哪些資料
print("\n=== 測試集句子展示 ===")
print("以下是測試集中的前 50 個句子：\n")

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

for i in range(50):
    sentence = test_dataset[i]["sentence"]
    label = test_dataset[i]["label"]
    print(f"[{i}] {sentence}")
    print(f"    真實標籤：{label_map[label]}\n")


=== 測試集句子展示 ===
以下是測試集中的前 50 個句子：

[0] TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .
    真實標籤：Positive

[1] STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
    真實標籤：Positive

[2] Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004 .
    真實標籤：Positive

[3] Lifetree was founded in 2000 , and its revenues have risen on an average by 40 % with margins in late 30s .
    真實標籤：Positive

[4] Nordea Group 's operating profit increased in 2010 by 18 percent year-on-year to 3.64 billion euros and total revenue by 3 percent to 9.33 billion euros .
    真實標籤：Positive

[5] Operating profit for the nine-month period increased from EUR3 .1 m and net sales increased fr

In [22]:
# 請從上面顯示的測試集句子中，選擇你想要測試的句子索引
#
# 選擇特定的句子索引
# 例如：test_indices = [0, 5, 10, 15, 19]
#
# 提示：
# - 可以選擇不同情緒的句子來測試
# - 觀察模型在哪些句子上預測正確，哪些預測錯誤
# - 思考為什麼模型會預測錯誤

test_indices = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499]  # 請修改這裡！選擇你想測試的句子索引

print(f"\n你選擇測試的句子索引：{test_indices}")
# ============================================================================

# 根據選擇的索引，取得測試句子
test_texts = [test_dataset[i]["sentence"] for i in test_indices]
true_labels = [test_dataset[i]["label"] for i in test_indices]

print("\n=== 開始預測 ===\n")

"""
預測流程說明：

1. 取得測試句子和真實標籤
   - 根據選擇的索引提取句子

2. Tokenization
   - 將句子轉換成模型輸入格式
   - return_tensors="pt" 返回 PyTorch tensor

3. 模型推理
   - 將 tokenized 資料輸入模型
   - 得到 logits（未標準化的分數）

4. 預測類別
   - 使用 argmax 取得分數最高的類別
   - 轉換回 CPU 以便後續處理

5. 結果展示
   - 顯示原始句子、真實標籤、預測標籤
   - 標記預測是否正確
   - 計算準確率
"""

# Tokenize 測試句子
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)
outputs = model(**test_encodings)

# 取得預測結果
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()  # 將結果轉回 CPU 以便處理

# 顯示預測結果
predicted_labels = [label_map[pred] for pred in preds]
true_labels_text = [label_map[label] for label in true_labels]

# 計算準確率
correct = sum([1 for true, pred in zip(true_labels_text, predicted_labels) if true == pred])
accuracy = correct / len(test_texts) * 100

print(f"你選擇的 {len(test_texts)} 個句子的預測結果：\n")

for i, (idx, text, true_label, pred_label) in enumerate(zip(test_indices, test_texts, true_labels_text, predicted_labels)):
    correct_mark = "✓ 正確" if true_label == pred_label else "✗ 錯誤"
    print(f"{i+1}. 測試集索引 [{idx}]")
    print(f"   句子：{text}")
    print(f"   真實標籤：{true_label}")
    print(f"   預測標籤：{pred_label}")
    print(f"   結果：{correct_mark}\n")

print(f"準確率：{correct}/{len(test_texts)} = {accuracy:.2f}%")


你選擇測試的句子索引：[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499]

=== 開始預測 ===

你選擇的 95 個句子的預測結果：

1. 測試集索引 [2]
   句子：Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004 .
   真實標籤：Positive
   預測標籤：Positive
   結果：✓ 正確

2. 測試集索引 [3]
   句子：Lifetree was founded in 2000 , and its revenues have risen on an average by 40 % with margins in late 30s .
   真實標籤：Positive
   預測標籤：Positive
   結果：✓ 正確

3. 測試集索引 [5]
   句子：Operating profit for the nine-month period increased from EUR3 .1 m and net sales increased from EUR61 .5 